# Part 3. More Fun With Word Vectors

[https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-3-more-fun-with-word-vectors](https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-3-more-fun-with-word-vectors)

<br>

# Code

The tutorial code for Part 3 lives [here](https://github.com/wendykan/DeepLearningMovies/blob/master/Word2Vec_BagOfCentroids.py). 

<br>

# Numeric Representations of Words

Now that we have a trained model with some semantic(의미있는) understanding of words, how should we use it? If you look beneath the hood, the Word2Vec model trained in Part 2 consists of a feature vector for each word in the vocabulary, stored in a `numpy` array called "syn0":

In [1]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec

model = Word2Vec.load("300features_40minwords_10context")

In [4]:
type(model.wv.syn0)

C:\Users\shkim\AppData\Local\conda\conda\envs\TF_VS_36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [5]:
model.wv.syn0.shape

C:\Users\shkim\AppData\Local\conda\conda\envs\TF_VS_36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(16490, 300)

The number of rows in syn0 is the number of words in the model's vocabulary, and the number of columns corresponds to the size of the feature vector, which we set in Part 2.  Setting the minimum word count to 40 gave us a total vocabulary of 16,492 words with 300 features apiece. Individual word vectors can be accessed in the following way:

In [6]:
model["flower"]

C:\Users\shkim\AppData\Local\conda\conda\envs\TF_VS_36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.03592819, -0.03085953,  0.05765823, -0.08588445, -0.06296363,
       -0.02721645,  0.02364479,  0.02526847,  0.00199798, -0.02929461,
       -0.07619712, -0.01337987, -0.10034997,  0.02408944, -0.11489791,
        0.06175002,  0.03637884, -0.02622486, -0.00348732,  0.07307983,
       -0.00288751,  0.02181118, -0.10311554, -0.04091304, -0.08759024,
        0.00267776, -0.02773638,  0.02627153,  0.08954636,  0.06634776,
        0.052514  ,  0.13716377, -0.03400363,  0.0015823 , -0.01543875,
        0.05707104,  0.04549051,  0.0119594 , -0.05463668, -0.01447129,
        0.03310588, -0.00459056, -0.06055202,  0.0118513 , -0.00734887,
       -0.07854102, -0.00864874,  0.08309419, -0.0052785 ,  0.12740564,
        0.01619201,  0.05162026,  0.06758543,  0.12929584,  0.01957222,
        0.05317652, -0.07101942, -0.06968176,  0.08475519,  0.02418363,
        0.12001555,  0.03276131,  0.11581071, -0.04504692,  0.06916741,
       -0.09433027, -0.0422213 , -0.03798748, -0.04629346,  0.00

... which returns a 1x300 numpy array.

<br>

# From Words To Paragraphs, Attempt 1: Vector Averaging

One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

The following code averages the feature vectors, building on our code from Part 2. 

In [13]:
import numpy as np # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features) :
    
    # Function to average all of the word vectors in a given paragraph
    
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,), dtype="float32")
    
    nwords = 0
    
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words :
        if word in index2word_set :
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
            
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features) :
    
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    
    # Initialize a counter
    counter = 0
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    
    # Loop through the reviews
    for review in reviews :
        
        # Print a status message every 1000th review
        if counter % 1000. == 0. :
            print("Review %d of %d" % (counter, len(reviews)))
            
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        
        # Increment the counter
        counter = counter + 1
        
    return reviewFeatureVecs

Now, we can call these functions to create average vectors for each paragraph. The following operations will take a few minutes:

In [14]:
import pandas as pd

# Read data from files
train = pd.read_csv("../input/labeledTrainData.tsv",
                    header=0,
                    delimiter="\t",
                    quoting=3)
test = pd.read_csv("../input/testData.tsv",
                   header=0,
                   delimiter="\t",
                   quoting=3)
unlabeled_train = pd.read_csv("../input/unlabeledTrainData.tsv",
                              header=0,
                              delimiter="\t",
                              quoting=3)

In [15]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [16]:
# Set values for various parameters
num_features = 300 # Word Vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsampling setting for frequent words

In [17]:
from tqdm import tqdm

In [18]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word removal.

clean_train_reviews = []

for review in tqdm(train["review"]) :
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print("Creating average feature vecs for test reviews")
clean_test_reviews = []

for review in tqdm(test["review"]) :
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:26<00:00, 950.51it/s]


Review 0 of 25000


C:\Users\shkim\AppData\Local\conda\conda\envs\TF_VS_36\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews


100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:24<00:00, 1035.48it/s]


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


Next, use the average paragraph vectors to train a random forest. Note that, as in Part 1, we can only use the labeled training reviews to train the model.

In [20]:
trainDataVecs.shape

(25000, 300)

In [21]:
testDataVecs.shape

(25000, 300)

In [22]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)

print("Fitting a random forest to labeled traininig data...")

forest = forest.fit(trainDataVecs, train["sentiment"])

# Test & extract results
result = forest.predict(testDataVecs)

# Write the test results
output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)

Fitting a random forest to labeled traininig data...


We found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points.

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "[tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial(상당한) improvement in performance.

<br>

# From Words to Paragraphs, Attempt 2: Clustering